# Battle of Neighborhoods: Mumbai

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
import geocoder

## Data scraping

Reading csv file into a pandas dataframe

In [3]:
df = pd.read_csv("Mumbai_pin.csv")
df.head()

,City,Post Office,Pin Code
0,Mumbai,August Kranti Marg,400036
1,Mumbai,Aarey Milk Colony,400065
2,Mumbai,Andheri (East),400069
3,Mumbai,Andheri (West),400058
4,Mumbai,Antop Hill,400037


Using geocoder to find out the geographical coordinates of the neighborhoods

In [4]:
address = 'Mumbai, IN'

geolocator = Nominatim(user_agent="mum_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


In [5]:
def get_latlng(Pin_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(Pin_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [6]:
coords = [ get_latlng(pin_Code) for pin_Code in df["Pin Code"].tolist() ]

In [7]:
lat_long = pd.DataFrame(coords, columns = ['latitude', 'longitude'])

lat_long.head()

,latitude,longitude
0,18.964005,72.807983
1,19.161085,72.884394
2,19.119298,72.851100
3,19.122935,72.840610
4,19.020313,72.868280


Populating the neighborhoods' dataframe with the geographical coordinates

In [8]:
df['latitude'] = lat_long['latitude']
df['longitude'] = lat_long['longitude']

df.head()

,City,Post Office,Pin Code,latitude,longitude
0,Mumbai,August Kranti Marg,400036,18.964005,72.807983
1,Mumbai,Aarey Milk Colony,400065,19.161085,72.884394
2,Mumbai,Andheri (East),400069,19.119298,72.851100
3,Mumbai,Andheri (West),400058,19.122935,72.840610
4,Mumbai,Antop Hill,400037,19.020313,72.868280


In [9]:
Lat = df['latitude'].mean()
Long = df['longitude'].mean()
mumbai_latlong = [Lat, Long]
mumbai_latlong

[19.038024435294627, 72.86260949874192]

## Data Visualization

Use folium package to visualize the neighborhoods from the dataframe 

In [10]:
map_mumbai = folium.Map(mumbai_latlong, zoom_start=10)

In [11]:
for lat, lng, neighborhood in zip(df['latitude'], df['longitude'], df['Post Office']):
    label = '{}, {}'.format(df, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

## Foursquare API

Foursquare credentials

In [12]:
CLIENT_ID = '0BGU3T2B50MKLZK0QNPR3GEMN155RTM0URK5YGRTITP530NI' # your Foursquare ID
CLIENT_SECRET = 'S10NMEK0LRWWU2GVNTBHRHLILDLV0BSRRPJABFGH5DCZ0LAZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Function for URL

In [13]:
def geturl(lat, long):
    limit = 100
    radius = 500
    url  = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    long,
    radius,
    limit)
    
    return url

Function to explore Mumbai neighborhoods

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    limit = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Running the above function and store it in a new dataframe

In [17]:
mumbai_venues = getNearbyVenues(names=df['Post Office'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

August Kranti Marg
Aarey Milk Colony
Andheri (East)
Andheri (West)
Antop Hill
Anu Shakti Nagar
B A R C
Ballard Estate
Bandra (East)
Bandra (West)
Bangur Nagar
Barve Nagar
Bhandup
Bhandup (East)
Bhavani Shankar Road
Mumbai Central
Mumbai G P O
Borivli (East)
Borivli (West)
Borivli HO
Chakala MIDC
Chembur
Chinch Bunder
Colaba
Council Hall
Cumballa Hill
Dadar
Dahisar
Delisle Road
Dharavi
F C I Mumbai
Ghatkopar (West)
Girgaon
Goregaon (East)
Goregaon (West)
Grant Road
Hutatma Chowk
I I T Mumbai
J B Nagar
Jacob Circle
Jogeshwari (East)
Jogeshwari (West)
Juhu
Kalbadevi
Kandivli (East)
Kandivli (West)
Khar
Kharodi
Kurla
Mahim
Malabar Hill
Malad (East)
Malad (West)
Mandpeshwar
Mandvi
Mantralaya
Marine Lines
Matunga
Mazgaon
Motilal Nagar
Mulund (East)
Mulund (West)
Mulund Colony
N I T I E
Nariman Point
Nehru Nagar
Pant Nagar
Parel
Poonam Ngr Jogeshwari (E)
Prabhadevi
Rajawadi
Rajbhavan
SEEPZ
Sahar
Saki Naka
Santacruz (East)
Santacruz (West)
Santacruz P&T Colony
Sewri
Shivaji Nagar (Kurla)
Sion


Analyzing neighborhoods with the venues

In [18]:
mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aarey Milk Colony,3,3,3,3,3,3
Agashi,13,13,13,13,13,13
Airoli Mode,14,14,14,14,14,14
Andheri (East),7,7,7,7,7,7
Andheri (West),7,7,7,7,7,7
Antop Hill,2,2,2,2,2,2
Anu Shakti Nagar,4,4,4,4,4,4
Arnala,13,13,13,13,13,13
August Kranti Marg,26,26,26,26,26,26


In [19]:
print('There are {} unique venue categories in Mumbai' .format(len(mumbai_venues['Venue Category'].unique())))

There are 189 unique venue categories in Mumbai


One hot encoding for the venue categories

In [20]:
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Zoo,ATM,Airport,Airport Lounge,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Theater,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
mumbai_onehot.shape

(1915, 189)

Finding frequencies for every venue category in each neighborhood

In [22]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped

,Neighborhood,Zoo,ATM,Airport,Airport Lounge,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Theater,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Wine Bar,Women's Store,Yoga Studio
0,Aarey Milk Colony,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,Agashi,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.076923,0.000000,0.0,0.000000,0.000000,0.0
2,Airoli Mode,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.071429,0.0,0.000000,0.000000,0.0
3,Andheri (East),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.142857,0.0,0.000000,0.000000,0.0
4,Andheri (West),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.142857,0.142857,0.0,0.000000,0.000000,0.0
5,Antop Hill,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.5,0.000000,0.000000,0.0,0.000000,0.000000,0.0
6,Anu Shakti Nagar,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
7,Arnala,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.076923,0.000000,0.0,0.000000,0.000000,0.0
8,August Kranti Marg,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
9,Balcum,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.076923,0.000000,0.0,0.000000,0.000000,0.0


In [23]:
num_top_venues = 5

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aarey Milk Colony----
                     venue  freq
0             Dance Studio  0.33
1    Performing Arts Venue  0.33
2              Event Space  0.33
3  New American Restaurant  0.00
4                 Mountain  0.00


----Agashi----
                venue  freq
0   Indian Restaurant  0.15
1                 Bar  0.15
2                Café  0.08
3       Train Station  0.08
4  Seafood Restaurant  0.08


----Airoli Mode----
                           venue  freq
0              Indian Restaurant  0.21
1                     Food Court  0.07
2             Italian Restaurant  0.07
3               Department Store  0.07
4  Vegetarian / Vegan Restaurant  0.07


----Andheri (East)----
                           venue  freq
0              Indian Restaurant  0.29
1              Indian Sweet Shop  0.14
2  Vegetarian / Vegan Restaurant  0.14
3                 Sandwich Place  0.14
4                      Juice Bar  0.14


----Andheri (West)----
                           venue  freq
0           

4               Zoo  0.00


----Ghatkopar (West)----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.17
1                     Smoke Shop  0.08
2                  Hot Dog Joint  0.08
3                    Coffee Shop  0.08
4           Fast Food Restaurant  0.08


----Gholvad----
                venue  freq
0   Indian Restaurant  0.15
1                 Bar  0.15
2                Café  0.08
3       Train Station  0.08
4  Seafood Restaurant  0.08


----Girgaon----
                  venue  freq
0           Snack Place  0.12
1             Juice Bar  0.12
2  Fast Food Restaurant  0.12
3           Coffee Shop  0.06
4             Multiplex  0.06


----Goregaon (East)----
                venue  freq
0                Café  0.25
1        Dance Studio  0.12
2   Indian Restaurant  0.12
3      Sandwich Place  0.12
4  Chinese Restaurant  0.12


----Goregaon (West)----
               venue  freq
0  Indian Restaurant  0.29
1        Pizza Place  0.14
2         Smoke Shop  0.14
3

                  venue  freq
0  Fast Food Restaurant  0.25
1     Indian Restaurant  0.08
2    Athletics & Sports  0.08
3                 Diner  0.08
4           Flea Market  0.08


----Mulund Colony----
               venue  freq
0     Ice Cream Shop  0.12
1              Trail  0.12
2           Mountain  0.12
3        Snack Place  0.12
4  Food & Drink Shop  0.12


----Mumbai Central----
                       venue  freq
0                      Hotel  0.18
1       Fast Food Restaurant  0.18
2                Pizza Place  0.09
3                 Smoke Shop  0.09
4  Middle Eastern Restaurant  0.09


----Mumbai G P O----
                venue  freq
0   Indian Restaurant  0.12
1                Café  0.12
2          Irani Cafe  0.12
3              Lounge  0.06
4  Chinese Restaurant  0.06


----Mumbra----
                venue  freq
0   Indian Restaurant  0.15
1                 Bar  0.15
2                Café  0.08
3       Train Station  0.08
4  Seafood Restaurant  0.08


----N A D Karanja----

                venue  freq
0   Indian Restaurant  0.15
1                 Bar  0.15
2                Café  0.08
3       Train Station  0.08
4  Seafood Restaurant  0.08


----Uttan----
                venue  freq
0   Indian Restaurant  0.15
1                 Bar  0.15
2                Café  0.08
3       Train Station  0.08
4  Seafood Restaurant  0.08


----Vajreshwari----
                venue  freq
0   Indian Restaurant  0.15
1                 Bar  0.15
2                Café  0.08
3       Train Station  0.08
4  Seafood Restaurant  0.08


----Vangam----
                venue  freq
0   Indian Restaurant  0.15
1                 Bar  0.15
2                Café  0.08
3       Train Station  0.08
4  Seafood Restaurant  0.08


----Vasai East I/E----
                     venue  freq
0           Ice Cream Shop   0.2
1  North Indian Restaurant   0.2
2         Business Service   0.2
3                   Lounge   0.2
4             Dance Studio   0.2


----Vasai Road East----
                venue  f

Adding the frequencies of the venue categories: Indian Restaurant and Chinese Restaurant

In [29]:
mumbai_rest = mumbai_grouped[["Neighborhood", "Indian Restaurant", "Chinese Restaurant"]]
mumbai_rest['IndoChinese Restaurant'] = mumbai_rest['Indian Restaurant'] + mumbai_rest['Chinese Restaurant']
mumbai_IndoChinese = mumbai_rest.drop(["Indian Restaurant", "Chinese Restaurant"],1)
mumbai_IndoChinese.head()

C:\Users\Lenovo\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Neighborhood,IndoChinese Restaurant
0,Aarey Milk Colony,0.000000
1,Agashi,0.230769
2,Airoli Mode,0.285714
3,Andheri (East),0.285714
4,Andheri (West),0.285714


## K-means Clustering

Run k-means clustering to classify Mumbai neighborhoods into 3 clusters

In [35]:
# set number of clusters
kclusters = 3

mumbai_clustering = mumbai_IndoChinese.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:]

array([1, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1])

Adding Cluster labels column with the neighborhood names

In [36]:
mumbai_merged = mumbai_IndoChinese.copy()

mumbai_merged["Cluster Labels"] = kmeans.labels_

In [37]:
mumbai_merged.head()

,Neighborhood,IndoChinese Restaurant,Cluster Labels
0,Aarey Milk Colony,0.000000,1
1,Agashi,0.230769,0
2,Airoli Mode,0.285714,0
3,Andheri (East),0.285714,0
4,Andheri (West),0.285714,0


In [38]:
mumbai_merged = mumbai_merged.join(df.set_index("Post Office"), on="Neighborhood")
mumbai_merged = mumbai_merged.drop(["City", "Pin Code"], 1)
print(mumbai_merged.shape)
mumbai_merged.head()

(147, 5)


,Neighborhood,IndoChinese Restaurant,Cluster Labels,latitude,longitude
0,Aarey Milk Colony,0.000000,1,19.161085,72.884394
1,Agashi,0.230769,0,18.940170,72.834830
2,Airoli Mode,0.285714,0,19.158562,72.994555
3,Andheri (East),0.285714,0,19.119298,72.851100
4,Andheri (West),0.285714,0,19.122935,72.840610


Sorting by Cluster Labels

In [39]:
mumbai_merged.sort_values(["Cluster Labels"], inplace=True)
mumbai_merged

,Neighborhood,IndoChinese Restaurant,Cluster Labels,latitude,longitude
73,Mahim,0.277778,0,19.037234,72.842440
119,Talasari,0.230769,0,18.940170,72.834830
74,Malabar Hill,0.166667,0,18.951180,72.798555
145,Wagle Industrial Estate,0.230769,0,18.940170,72.834830
72,Kurla,0.250000,0,19.076449,72.879810
71,Krishi Utpanna Bazar,0.250000,0,19.074753,73.021375
70,Kosbad Hill,0.230769,0,18.940170,72.834830
121,Tarapur,0.230769,0,18.940170,72.834830
122,Tarapur App,0.230769,0,18.940170,72.834830
123,Tarapur J/A,0.230769,0,18.940170,72.834830


Visualizing the clusters on the map of Mumbai

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['latitude'], mumbai_merged['longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 0 neighborhoods

In [45]:
cluster_0 = mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0]
cluster_0

,Neighborhood,IndoChinese Restaurant,Cluster Labels,latitude,longitude
73,Mahim,0.277778,0,19.037234,72.842440
119,Talasari,0.230769,0,18.940170,72.834830
74,Malabar Hill,0.166667,0,18.951180,72.798555
145,Wagle Industrial Estate,0.230769,0,18.940170,72.834830
72,Kurla,0.250000,0,19.076449,72.879810
71,Krishi Utpanna Bazar,0.250000,0,19.074753,73.021375
70,Kosbad Hill,0.230769,0,18.940170,72.834830
121,Tarapur,0.230769,0,18.940170,72.834830
122,Tarapur App,0.230769,0,18.940170,72.834830
123,Tarapur J/A,0.230769,0,18.940170,72.834830


Cluster 1 neighborhoods

In [48]:
cluster_1 = mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1]
cluster_1

,Neighborhood,IndoChinese Restaurant,Cluster Labels,latitude,longitude
113,Santacruz P&T Colony,0.000000,1,19.087855,72.867165
110,Sandoz Baug,0.062500,1,19.256540,72.983650
120,Tank Road,0.000000,1,18.986693,72.844630
144,Wadala,0.000000,1,19.014590,72.855370
142,Vile Parle (West),0.117647,1,19.105737,72.841250
126,Tilak Nagar,0.111111,1,19.068165,72.899838
111,Santacruz (East),0.105263,1,19.080565,72.851489
112,Santacruz (West),0.000000,1,19.083779,72.833930
138,Vesava (Versova),0.000000,1,19.144755,72.803636
124,Thane (East),0.000000,1,19.178750,72.974521


Cluster 2 neighborhoods

In [50]:
cluster_2 = mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2]
cluster_2

,Neighborhood,IndoChinese Restaurant,Cluster Labels,latitude,longitude
6,Anu Shakti Nagar,0.500000,2,19.033945,72.925200
60,Kalwa,0.500000,2,19.183108,72.992143
40,F C I Mumbai,1.000000,2,19.022904,72.896845
77,Mandvi,0.466667,2,18.952760,72.834560
78,Manor,0.666667,2,19.721010,72.933310
